In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT

datapath = '../input/predict_probs_resnet50.h5'
BATCHES_COUNT = 12091

In [2]:
%%time
store = pd.HDFStore(datapath)
keys = store.keys()
print('{} tables in storage'.format(len(keys)))

12091 tables in storage
CPU times: user 38.6 s, sys: 400 ms, total: 39 s
Wall time: 39 s


In [3]:
assert len(keys) == BATCHES_COUNT

preds = []
cur_id = None
cur_sum = None

for key in tqdm(keys): 
    table = store.select(key)
    # table.drop(table[table.img_num != 0.].index, axis=0, inplace=True)
    assert table.shape[0] % 10 == 0
    for i in range(int(table.shape[0] / 10)):
        product_id = table.pr_id.iloc[10*i]
        new_sum = table[10*i : 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        
        if cur_id == product_id:
            assert cur_sum is not None
            cur_sum += new_sum
            continue
        elif cur_id is not None:
            assert cur_sum is not None
            id_ = int(cur_sum.idxmax()) - 1
            cat = LABEL_TO_CAT[id_]
            preds.append((int(cur_id), cat))
            
        cur_id = product_id
        cur_sum = new_sum
        
if cur_id is not None:
    id_ = int(cur_sum.idxmax()) - 1
    cat = LABEL_TO_CAT[id_]
    preds.append((int(cur_id), cat))
            
preds_df = pd.DataFrame(data=preds, columns=['_id','category_id'])
print('Dataframe shape', preds_df.shape)
preds_df.to_csv('preds_resnet50_all_imgs.csv', index=False)

100%|██████████| 12091/12091 [2:12:12<00:00,  1.52it/s] 


Dataframe shape (1769197, 2)


In [5]:
preds_df.head()

,_id,category_id
0,14,1000010653
1,49,1000011448
2,85,1000014276
3,86,1000002805
4,93,1000010653


In [9]:
no_dupls = preds_df.drop_duplicates(subset='_id', inplace=False)
print(no_dupls.shape)
no_dupls.head()

(1768182, 2)


,_id,category_id
0,14,1000010653
1,49,1000011448
2,85,1000014276
3,86,1000002805
4,93,1000010653


In [10]:
no_dupls.to_csv('preds_resnet50_all_imgs_correct.csv', index=False)